
# TP – Prédire le taux d’utilisation CPU d’un serveur à partir d’indicateurs réseau

**Objectif pédagogique :** construire, expliquer et évaluer un modèle de **régression** qui anticipe la **charge CPU** d’un serveur à partir d’indicateurs réseau, comme on le ferait dans un **SOC (Security Operations Center)** pour détecter des surcharges (ex. DDoS) et comportements anormaux.

## Plan
1. Contexte de l’étude  
2. Importations & données (simulation réaliste)  
3. Exploration & pré‑traitement  
   - Aperçu & valeurs manquantes  
   - Analyse de corrélation  
   - Encodage (rappel) & Standardisation  
4. Train/Test split  
5. Sélection séquentielle de variables (**SequentialFeatureSelector**)  
6. Ajustement du modèle (régression linéaire) et évaluation  
7. Amélioration par interactions (**PolynomialFeatures**) et ré‑évaluation  
8. Recommandations


In [ ]:

# === IMPORTS ===
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.feature_selection import SequentialFeatureSelector



## 2) Données : simulation réaliste (logs réseau)
Dans un contexte SOC, on observe par **intervalle de temps** :
- `tcp_connections` : nb de connexions TCP actives  
- `data_volume_MB` : volume transmis (Mo)  
- `packet_loss_rate` : taux de perte de paquets (%)  
- `ids_alerts` : nb d’alertes IDS  
- `cpu_load` : charge CPU (cible à prédire)

> Ici, on **simule** un jeu de données cohérent avec du bruit pour pratiquer la modélisation.



# Génération de données synthétiques (reproductibles)
np.random.seed(42)
n = 500

data = pd.DataFrame({
    "tcp_connections": np.random.randint(50, 2000, n),
    "data_volume_MB": np.random.uniform(100, 10000, n),
    "packet_loss_rate": np.random.uniform(0, 5, n),   # en %
    "ids_alerts": np.random.randint(0, 50, n)
})

# Relation linéaire + bruit
noise = np.random.normal(0, 5, n)
data["cpu_load"] = (
    0.02 * data["tcp_connections"]
    + 0.003 * data["data_volume_MB"]
    + 1.5 * data["packet_loss_rate"]
    + 0.3 * data["ids_alerts"]
    + 0.000005 * data["data_volume_MB"]*data["tcp_connections"]
    + noise
)

data.head()


data.to_csv("G:\Mon Drive\ADD\CS2C-Cybersecurity Data Analytics\TDs-TPs\cpu_load_data_.csv", index=False)

In [ ]:
data=pd.read_csv("cpu_load_data_.csv")
data.columns


## 3) Exploration & pré‑traitement
### 3.1 Aperçu général & qualité des données
- Vérifier formats, valeurs manquantes, statistiques descriptives.


In [ ]:

data.info()


In [ ]:

data.describe().T



### 3.2 Analyse de corrélation
- But : estimer la **force des relations linéaires** entre variables.  
- Outil : matrice des corrélations de Pearson.  
> **Rappel :** La corrélation ≠ causalité ; elle guide le choix de variables et l'interprétation.


In [ ]:

# Matrice de corrélation
corr = data.corr(numeric_only=True)

# Visualisation avec matplotlib (pas de seaborn, 1 plot par figure, couleurs par défaut)
plt.figure(figsize=(6,5))
im = plt.imshow(corr.values, aspect='auto', interpolation='nearest')
plt.colorbar(im, fraction=0.046, pad=0.04)
plt.xticks(range(len(corr.columns)), corr.columns, rotation=45, ha='right')
plt.yticks(range(len(corr.columns)), corr.columns)
plt.title("Matrice de corrélation (Pearson)")
plt.tight_layout()
plt.show()

corr



### 3.3 Encodage des variables qualitatives (rappel)
Si vous aviez des colonnes **catégorielles** (`type_proto`, `zone`, etc.), utilisez `pd.get_dummies(..., drop_first=True)` ou `OneHotEncoder`.  
Ici, toutes les variables sont **numériques**, donc pas d’encodage à faire.

### 3.4 Standardisation
- Les variables sont sur des **échelles différentes** (Mo, %, compte…).  
- On **standardise** pour améliorer la stabilité de l’entraînement.


In [ ]:

X = data.drop(columns=["cpu_load"])
y = data["cpu_load"].copy()

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Garder les noms de features pour le suivi
feature_names = X.columns.tolist()

X.head()



## 4) Train/Test split
On sépare les données pour **évaluer la généralisation** du modèle.


In [ ]:

X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42
)
X_train.shape, X_test.shape



## 5) Sélection séquentielle de variables (**SequentialFeatureSelector**)
**Idée :** choisir automatiquement un sous‑ensemble pertinent de variables pour limiter la complexité et le sur‑ajustement.

- `direction="forward"` : on **ajoute** les variables les plus utiles une à une  
- `n_features_to_select=3` (modifiable pour expérimenter)


In [ ]:

linreg = LinearRegression()
sfs = SequentialFeatureSelector(linreg, n_features_to_select=3, direction="forward")
sfs.fit(X_train, y_train)

support_mask = sfs.get_support()
selected_features = [name for name, keep in zip(feature_names, support_mask) if keep]
selected_features



## 6) Ajustement du modèle linéaire & évaluation
On ajuste la **régression linéaire** avec les **variables sélectionnées**, puis on évalue sur le **jeu de test**.


In [ ]:

# Filtrer les colonnes sélectionnées
X_train_sfs = X_train[:, support_mask]
X_test_sfs  = X_test[:, support_mask]

linreg.fit(X_train_sfs, y_train)
y_pred = linreg.predict(X_test_sfs)

mae  = mean_absolute_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred) ** 0.5
r2   = r2_score(y_test, y_pred)

print(f"MAE  : {mae:.2f}")
print(f"RMSE : {rmse:.2f}")
print(f"R²   : {r2:.3f}")


In [ ]:

# Nuage réel vs prédit (1 figure, pas de style/couleurs spécifiques)
plt.figure()
plt.scatter(y_test, y_pred)
min_v = min(y_test.min(), y_pred.min())
max_v = max(y_test.max(), y_pred.max())
plt.plot([min_v, max_v], [min_v, max_v], linestyle="--")
plt.xlabel("Valeurs réelles (CPU load)")
plt.ylabel("Valeurs prédites")
plt.title("Régression linéaire – Réel vs Prédit")
plt.tight_layout()
plt.show()



## 7) Amélioration : interactions (**PolynomialFeatures**)
Les **interactions** (produits entre variables) modélisent des effets combinés (ex. trafic élevé × pertes de paquets).  
Ici, on génère toutes les **caractéristiques polynomiales de degré 2** (sans biais), puis on ré‑entraîne un modèle linéaire.


In [ ]:
poly = PolynomialFeatures(degree=2, include_bias=False)
X_poly = poly.fit_transform(X_scaled)

X_train_p, X_test_p, y_train_p, y_test_p = train_test_split(
    X_poly, y, test_size=0.2, random_state=42
)

linreg_poly = LinearRegression()
linreg_poly.fit(X_train_p, y_train_p)
y_pred_p = linreg_poly.predict(X_test_p)

mae_p  = mean_absolute_error(y_test_p, y_pred_p)
rmse_p = mean_squared_error(y_test_p, y_pred_p) ** 0.5
r2_p   = r2_score(y_test_p, y_pred_p)

print(f"MAE (poly)  : {mae_p:.2f}")
print(f"RMSE (poly) : {rmse_p:.2f}")
print(f"R² (poly)   : {r2_p:.3f}")

In [ ]:
# Analyse des coefficients du modèle polynomial
pd.DataFrame({"Variables": poly.get_feature_names_out(input_features=X.columns).tolist(), 
             "Coef": linreg_poly.coef_}).sort_values(by="Coef", key=abs, ascending=False)

In [ ]:
# Nuage réel vs prédit – modèle avec interactions
plt.figure()
plt.scatter(y_test_p, y_pred_p)
min_v = min(y_test_p.min(), y_pred_p.min())
max_v = max(y_test_p.max(), y_pred_p.max())
plt.plot([min_v, max_v], [min_v, max_v], linestyle="--")
plt.xlabel("Valeurs réelles (CPU load)")
plt.ylabel("Valeurs prédites")
plt.title("Modèle avec interactions (PolynomialFeatures) – Réel vs Prédit")
plt.tight_layout()
plt.show()


## 8) Recommandations
- **Interprétation des coefficients** : examiner l’influence de chaque indicateur pour expliquer la charge CPU (utile en SOC).  
- **Interactions** : si la performance s’améliore, conserver quelques interactions pertinentes plutôt que toutes (risque de sur‑ajustement).  
- **Modèles alternatifs** : tester des approches non linéaires (`KNN`, `RandomForestRegressor`, `XGBoost`).   
- **Validation plus robuste** : utiliser une **validation croisée** et un **pipeline** scikit‑learn (scaler + sélection + modèle).


## 9) Travail à faire KNN (`KNeighborsRegressor()`)

1. Construisez un pipeline complet combinant les étapes suivantes :

    - Standardisation des variables d’entrée (`StandardScaler`)
    - Génération de caractéristiques non linéaires (`PolynomialFeatures`)
    - Sélection automatique de variables (`SequentialFeatureSelector`)
    - Modèle final de régression (`KNeighborsRegressor`)

2. Définissez ensuite une `grille` d’hyperparamètres  :

    - "n_features_to_select": [2, 3, 4, 5] # pour tester différents nombres de variables sélectionnées
    - "n_neighbors": [3, 5, 7, 9, 11, 13, 15]

3. Mettez en place un `GridSearchCV` pour identifier la combinaison optimale des hyperparamètres en utilisant la validation croisée et le score R²

In [ ]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import r2_score

base_knn = KNeighborsRegressor()

pipe = Pipeline([
    ("scaler", StandardScaler()),
        # Génération de features polynomiales
    ("poly", PolynomialFeatures(
        degree=2,
        include_bias=False
    )),
    ("sfs", SequentialFeatureSelector(
        estimator=base_knn,
        n_features_to_select=3,     # valeur par défaut (sera ajustée par la grille)
        direction="forward",
        scoring="r2",
        cv=5
    )),
    ("knn", KNeighborsRegressor())
])

param_grid = {
    "sfs__n_features_to_select": [2, 3, 4, 5],    
    "knn__n_neighbors": [3, 5, 7, 9, 11, 13, 15],
}

grid = GridSearchCV(
    estimator=pipe,
    param_grid=param_grid,
    scoring="r2",
    cv=5,
    verbose=1
)

In [ ]:
grid.fit(X_train, y_train)

print("Meilleurs hyperparamètres :")
print(grid.best_params_)
print("Meilleur R² (CV) :", grid.best_score_)

In [ ]:
sfs_best = grid.best_estimator_.named_steps["sfs"]
support = sfs_best.get_support()
selected_features = X.columns[support]
print("Features sélectionnées :", selected_features)

In [ ]:
# Évaluation test
# =========================
y_pred = grid.predict(X_test)
print("R² sur test :", r2_score(y_test, y_pred))

# Voir les features sélectionnées par SFS
sfs_best = grid.best_estimator_.named_steps["sfs"]
support = sfs_best.get_support()
print("Nombre de features polynomiales sélectionnées :", support.sum())

In [ ]:
best_pipe = grid.best_estimator_          # le meilleur pipeline trouvé
sfs = best_pipe.named_steps["sfs"]        # étape SFS du pipeline
poly = best_pipe.named_steps["poly"]
support = sfs.get_support()               # masque booléen des features sélectionnées

# X était un DataFrame avec des colonnes nommées
original_feature_names = X.columns

# Noms des features polynomiales (après PolynomialFeatures)
poly_feature_names = poly.get_feature_names_out(original_feature_names)

# Appliquer le masque du SFS sur ces noms
selected_poly_features = poly_feature_names[support]

print("Variables retenues dans le modèle :", selected_poly_features)


In [ ]:
import matplotlib.pyplot as plt

# Résultats du GridSearch
results = pd.DataFrame(grid.cv_results_)

# Extraire n_features_to_select et le mean_test_score
n_features = results["param_sfs__n_features_to_select"].astype(int)
scores = results["mean_test_score"]

plt.plot(n_features, scores, marker="o")
plt.xlabel("Nombre de variables sélectionnées (n_features_to_select)")
plt.ylabel("Score R² moyen (CV)")
plt.title("Impact du nombre de variables sur les performances du modèle")
plt.grid(True)
plt.show()


En fait, on faisait varier  knn__n_neighbors, donc pour un même n_features_to_select, on a plusieurs lignes dans cv_results_.

Pour chaque valeur de n_features_to_select, on prend le meilleur R² parmi toutes les autres combinaisons.

In [ ]:
results = pd.DataFrame(grid.cv_results_)

# S'assurer que c'est bien des entiers
results["param_sfs__n_features_to_select"] = results["param_sfs__n_features_to_select"].astype(int)

# Regrouper par n_features_to_select et prendre le meilleur score pour chacun
grouped = results.groupby("param_sfs__n_features_to_select")["mean_test_score"].max()

# Récupérer x et y
n_features = grouped.index.values
best_scores = grouped.values

plt.plot(n_features, best_scores, marker="o")
plt.xlabel("Nombre de variables sélectionnées (n_features_to_select)")
plt.ylabel("Meilleur score R² moyen (CV)")
plt.title("R² (CV) en fonction du nombre de variables sélectionnées")
plt.grid(True)
plt.show()


L'écart de performances ontenu par 3 et 5 variables ne semble pas significatif